In [9]:
# Turn off annoying ipykernel warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tableone import TableOne
from sklearn import linear_model
from patsy import dmatrices
import statsmodels.api as sm
import statsmodels.formula.api as smf

# New Onset Dosing in T1D

In [11]:
# Data import and cleaning
wd = '/Volumes/som/PEDS/RI Biostatistics Core/Shared/Shared Projects/Laura/BDC/Projects/Kimber Simmons/New Onset Dosing/Data_Cleaned/'
df = pd.read_csv(wd+'new_onset_doses.csv',na_values='.',skipinitialspace = True)
df.set_index(['MRN'],inplace=True)
# Remove T2D
df = df[df['Diabetes Type']=="T1D"]
# Format columns
# Convert all ages to months
df['Age'] = [int(a.split()[0])*12 if 'year' in a else int(a.split()[0]) for a in df['Age']]
# Categorize bicarb
bicarb_cat=[]
for b in df['bicarb']:
    if '<' in str(b):
        bicarb_cat.append('Severe')
    elif float(b) >= 5 and float(b) < 10:
        bicarb_cat.append('Moderate')
    elif float(b) >= 10:
        bicarb_cat.append('Mild')
    else:
        bicarb_cat.append(np.nan)
df['bicarb_cat'] = bicarb_cat
# Replace detection limit of bicarb with half the value
df['bicarb'] = [2.5 if '<' in str(b) and '5' in str(b) else 1.5 if '<' in str(b) and '3' in str(b) else float(b) for b in df['bicarb']]
# Convert to numeric
df['week1_carbbreakfast']=pd.to_numeric(df['week1_carbbreakfast'],errors='coerce')
df['week1_carblunch']=pd.to_numeric(df['week1_carblunch'],errors='coerce')
df['week1_carbdinner']=pd.to_numeric(df['week1_carbdinner'],errors='coerce')
# Combine levels of race
race = []
for r in df['Race']:
    if '\n' in str(r) or 'more than one' in str(r).lower():
        race.append('Multiracial')
    elif 'white' in str(r).lower():
        race.append('White')
    elif 'asian' in str(r).lower():
        race.append('Asian')
    elif 'black' in str(r).lower():
        race.append('Black/African American')
    elif 'native' in str(r).lower():
        race.append('Native American/Pacific Islander') 
    else: 
        race.append('Unknown/Not Reported')
for e in range(0,df.shape[0]):
    if df['Ethnicity'].reset_index(drop=True).iloc[e] == 'Hispanic or Latino [1]' and race[e] != 'Multiracial':
        race[e] = 'Hispanic'
df['Race'] = race
# Public vs. private insurance
# Public – Medicaid, TriCare, CHP, Indian Health Service, Denver Health (Medicaid)
# Private – Aetna, Anthem, Cigna, UHC, Kaiser, Cofinity, Christian Care
# Self-pay
public = ['medicaid','tricare','chp','indian','denver health']
private = ['aetna','anthem','cigna','uhc','kaiser','cofinity',
           'christian','commercial','friday','samaritan','rocky mountain','liberty','assurant']
insurance = []
for i in df['Insurance']:
    ins = str(i).lower()
    if 'Private' in ['Private' for p in private if p in ins]:
        insurance.append('Private')
    elif 'Public' in ['Public' for p in public if p in ins]: 
        insurance.append('Public')
    elif 'self' in ins:
        insurance.append('Self-pay')
    else: 
        insurance.append('Unknown/Not Reported')
df['Insurance'] = insurance
# Write to CSV
df.to_csv(wd+"cleaned_data.csv")

## Table 1: Participant Characteristics at Visit 1

In [14]:
# Make table 1
cols = ['Age','Sex','puberty_yn','Race','Insurance',
       'Initial_A1c','Hospitalization','DKA','pH','bicarb','bicarb_cat','Diabetes Type','bmi_onset','Wt_onset']
# Print
t1 = TableOne(df,columns=cols,display_all=True)
t1

Missing       Overall
n                                                                         336
Age, mean (SD)                                                0  117.4 (52.7)
Sex, n (%)             Female                                 0    157 (46.7)
                       Male                                        179 (53.3)
puberty_yn, n (%)      no                                     0    182 (54.2)
                       yes                                         154 (45.8)
Race, n (%)            Asian                                  0       4 (1.2)
                       Black/African American                        12 (3.6)
                       Hispanic                                     45 (13.4)
                       Multiracial                                   17 (5.1)
                       Native American/Pacific Islander               5 (1.5)
                       Unknown/Not Reported                         40 (11.9)
                       White                                       213 (63.4)
Insurance, n (%)       Private                                0    198 (58.9)
                       Public                                      122 (36.3)
                       Self-pay                                       7 (2.1)
                       Unknown/Not Reported                           9 (2.7)
Initial_A1c, mean (SD)                                        4    12.1 (2.7)
Hospitalization, n (%) No                                     5     44 (13.3)
                       Yes                                         287 (86.7)
DKA, n (%)             No                                     5    161 (48.6)
                       Yes                                         170 (51.4)
pH, mean (SD)                                                66     7.2 (0.3)
bicarb, mean (SD)                                            60    13.9 (7.3)
bicarb_cat, n (%)      Mild                                  74    179 (68.3)
                       Moderate                                     64 (24.4)
                       Severe                                        19 (7.3)
Diabetes Type, n (%)   T1D                                    0   336 (100.0)
bmi_onset, mean (SD)                                         23    17.8 (4.4)
Wt_onset, mean (SD)                                          15   37.4 (19.4)

# Methods
All analyses were performed using Python version 3.9.5 and R version 4.1.0. Model selection was based on 10-fold cross validation with hierarchical group-lasso regularization as implemented in the glinternet R package, version 1.0.11. Model selection was performed on variables: Age, Sex, Race/Ethnicity, Initial_A1c, DKA, and Wt_onset and all interactions with Average_TDD as the outcome. Bicarb was categorized as a  into severe (< 5), moderate (5 - 10), and mild (> 10). Race and Ethnicity were combined per Kimber's 6/22 email.

Next, a linear model was re-fit using the statsmodels library (version 0.12.2). A linear mixed model with random effect for provider produced convergence warnings and was not much better than a simple linear model. 

## Model Selection with ElasticNet

In [15]:
# Get model matrices
y,X = dmatrices('Average_TDD ~ Age*Sex+Age*Race+Age*Initial_A1c+Age*DKA+Age*Wt_onset',data = df)
# EN cross validation
regr = linear_model.ElasticNetCV(cv=10, random_state=1017)
cv = regr.fit(X, y.ravel())
# Print terms where coefficient is not 0
# [X.design_info.column_names[i] for i in range(0,cv.coef_.shape[0]) if cv.coef_[i] != 0]
# For testing interaction terms with the Lasso, use the glinternet R package

In [16]:
# Fit simple model
md = smf.ols("Average_TDD ~ Age+Initial_A1c*Wt_onset+Sex*DKA",data=df,missing='drop')
mdf = md.fit()
print(mdf.summary())

                            OLS Regression Results                            
Dep. Variable:            Average_TDD   R-squared:                       0.366
Model:                            OLS   Adj. R-squared:                  0.349
Method:                 Least Squares   F-statistic:                     22.09
Date:                Wed, 21 Jul 2021   Prob (F-statistic):           1.66e-23
Time:                        11:05:57   Log-Likelihood:                -1108.3
No. Observations:                 276   AIC:                             2233.
Df Residuals:                     268   BIC:                             2262.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 41

The output above is interpreted the same way as simple multiple regression (i.e. without random effects), with the "Coef." column representing the beta values. So, for each one unit increase in age (in months), Average_TDD increases by 0.091 on average while holding the other variables constant. These coefficients can be multiplied and keep the same interpretation, so for each 1 year increase in age, Average_TDD increases by 0.093 * 12 = 1.116 on average.

Model selection using regularization and cross-validation may retain some variables even though they are not statistically significant. These variables may not have a significant p value, but they help increase the model's predictive value.

## BMI at onset

In [17]:
md = smf.ols("Average_TDD ~ bmi_onset",data=df,missing='drop')
mdf = md.fit()
print(mdf.summary())

                            OLS Regression Results                            
Dep. Variable:            Average_TDD   R-squared:                       0.104
Model:                            OLS   Adj. R-squared:                  0.101
Method:                 Least Squares   F-statistic:                     31.85
Date:                Wed, 21 Jul 2021   Prob (F-statistic):           4.13e-08
Time:                        11:06:00   Log-Likelihood:                -1162.7
No. Observations:                 277   AIC:                             2329.
Df Residuals:                     275   BIC:                             2337.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     12.4627      4.325      2.881      0.0

## Insurance

In [18]:
md = smf.ols("Average_TDD ~ Insurance",data=df,missing='drop')
mdf = md.fit()
print(mdf.summary())

                            OLS Regression Results                            
Dep. Variable:            Average_TDD   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                    0.2149
Date:                Wed, 21 Jul 2021   Prob (F-statistic):              0.886
Time:                        11:06:02   Log-Likelihood:                -1253.1
No. Observations:                 295   AIC:                             2514.
Df Residuals:                     291   BIC:                             2529.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Interc